# PROJECT 3 (PART 1)
** CLASSIFYING JAPAN TRAVEL POST FROM JAPAN LIFE POST IN REDDIT.COM**
### -PREPROCESSING  

### PROBLEM STATEMENT
Classifying whether the post coming from japan travel or japan life Subreddit. This classification is done to improve customer satisfaction and to avoid missing of any information related to a particular post as they are poster under wrong subreddit. Reddit will be able to move the post under right category using this classification model.

In [1]:
# importing the libraries used in the coding
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
%matplotlib inline

### Data Cleaning

In [2]:
# maximum column display is increased to 999
pd.set_option('display.max_columns',999)

In [3]:
# reading scraped Japan trip csv with pandas
df1=pd.read_csv('..\datasets\Japan_trip.csv')

In [4]:
# checking the size of the data frame
df1.shape

(1498, 103)

In [5]:
# Checking the content of top 2 rows in the dataframe
df1.head(2)

,all_awardings,allow_live_comments,approved_at_utc,approved_by,archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,awarders,banned_at_utc,banned_by,can_gild,can_mod_post,category,clicked,content_categories,contest_mode,created,created_utc,discussion_type,distinguished,domain,downs,edited,gilded,gildings,hidden,hide_score,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,likes,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media,media_embed,media_only,mod_note,mod_reason_by,mod_reason_title,mod_reports,name,no_follow,num_comments,num_crossposts,num_reports,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,quarantine,removal_reason,report_reasons,saved,score,secure_media,secure_media_embed,selftext,selftext_html,send_replies,spoiler,steward_reports,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,total_awards_received,ups,url,user_reports,view_count,visited,whitelist_status,wls
0,[],True,NaN,NaN,False,JapanTravelBot,NaN,NaN,green,[],NaN,Be nice to me! I'm just a helpful bot!,dark,text,t2_3gj5jaqw,False,[],NaN,NaN,False,False,NaN,False,NaN,False,1.569330e+09,1.569301e+09,NaN,moderator,self.JapanTravel,0,False,0,{},False,False,d8ii90,False,False,False,False,True,True,False,NaN,NaN,one,[],NaN,Itinerary,dark,text,False,NaN,{},False,NaN,NaN,NaN,[],t3_d8ii90,False,518,0,NaN,False,all_ads,/r/JapanTravel/comments/d8ii90/rjapantravel_to...,False,self,{'images': [{'source': {'url': 'https://extern...,6,False,NaN,NaN,False,44,NaN,{},Are you traveling to Japan this month? Want to...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",False,False,[],True,JapanTravel,t5_2uylr,r/JapanTravel,1016576,public,NaN,self,NaN,NaN,/r/JapanTravel Tourist Meet-Up Megathread - tr...,0,44,https://www.reddit.com/r/JapanTravel/comments/...,[],NaN,False,all_ads,6
1,[],True,NaN,NaN,False,mithdraug,NaN,NaN,NaN,[],NaN,NaN,NaN,text,t2_j6bs8r,False,[],NaN,NaN,False,False,NaN,False,NaN,False,1.571150e+09,1.571122e+09,NaN,moderator,self.JapanTravel,0,1571289303.0,0,{},False,False,di3vbv,False,False,False,False,True,True,False,NaN,NaN,two,[],NaN,Question,dark,text,False,NaN,{},False,NaN,NaN,NaN,[],t3_di3vbv,False,184,0,NaN,False,all_ads,/r/JapanTravel/comments/di3vbv/info_status_typ...,False,NaN,NaN,6,False,NaN,NaN,False,188,NaN,{},Please post any questions regarding the damage...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,False,[],True,JapanTravel,t5_2uylr,r/JapanTravel,1016576,public,NaN,self,NaN,NaN,[INFO STATUS] Typhoon Hagibis damage mega-thread.,0,188,https://www.reddit.com/r/JapanTravel/comments/...,[],NaN,False,all_ads,6


In [6]:
# Extracting the necessary columns and renaming the dataframe
japan_travel_df=df1.loc[:,['id','selftext','subreddit','title']]

In [7]:
# checking the dataframe
japan_travel_df.head()

,id,selftext,subreddit,title
0,d8ii90,Are you traveling to Japan this month? Want to...,JapanTravel,/r/JapanTravel Tourist Meet-Up Megathread - tr...
1,di3vbv,Please post any questions regarding the damage...,JapanTravel,[INFO STATUS] Typhoon Hagibis damage mega-thread.
2,djdcfh,"Hi everyone, I cannot believe how late I'm pos...",JapanTravel,"Last-minute itinerary check: 10/19 - 10/28, To..."
3,dj6q70,English News: [https://www.tellerreport.com/li...,JapanTravel,PSA: parts of JR East Yamanote and Keihin–Toho...
4,djjsvc,"Hey, so me and my buddy are planning a trip so...",JapanTravel,Me and my travel-buddy really want to catch a ...


In [8]:
# checking the duplicates
japan_travel_df.duplicated().sum()

502

##### There are 502 duplicate posts

In [9]:
japan_travel_df[japan_travel_df['selftext'].isnull()]

,id,selftext,subreddit,title
378,dc7fl0,NaN,JapanTravel,Anyone viewed Mt Fuji from Koyo-Dai or Sanko-D...
382,dc3f22,NaN,JapanTravel,Is Sushi Daiwa worth it alone for the trip to ...
417,db2bfn,NaN,JapanTravel,Anyone recommend the whiskey distillery near t...
679,d74rc7,NaN,JapanTravel,Teamlab - I’m disabled with carer. Is the care...
861,d3lbug,NaN,JapanTravel,Anyone know how I can get tickets to the wbsc ...
1371,dc7fl0,NaN,JapanTravel,Anyone viewed Mt Fuji from Koyo-Dai or Sanko-D...
1375,dc3f22,NaN,JapanTravel,Is Sushi Daiwa worth it alone for the trip to ...
1410,db2bfn,NaN,JapanTravel,Anyone recommend the whiskey distillery near t...


In [10]:
# fill with single space for NaN to retain the title after title and selftest concatenation
japan_travel_df['selftext']=japan_travel_df['selftext'].fillna(' ')

In [11]:
# deleting the duplicate posts
japan_travel_df=japan_travel_df.drop_duplicates(['id'], keep='last')

In [12]:
# checking the size
japan_travel_df.shape

(996, 4)

##### We have 996 rows which are good enough for our classification.

In [13]:
# Adding title and post content together. So that we can classify also based on title
japan_travel_df['content']=japan_travel_df['title']+" "+japan_travel_df['selftext']


In [14]:
# removing all colums other than content to be classified and its category for training and testing
japan_travel_df=japan_travel_df.drop(columns=['title','selftext','id'])
japan_travel_df=japan_travel_df.reindex(columns=['content','subreddit'])
japan_travel_df.head()

,content,subreddit
4,Me and my travel-buddy really want to catch a ...,JapanTravel
13,Ishigaki/Okinawa Just wondering where the best...,JapanTravel
15,PSA: Tobu-Nikko service resumption Anyone else...,JapanTravel
505,Did I Book the Osaka Cup Ramen Museum Correctl...,JapanTravel
506,Tokyo motorcycle route Thinking of renting a m...,JapanTravel


In [15]:
# Reading the 2nd category Japan life from the scrapped csv file into a pandas dataframe
df2=pd.read_csv('..\datasets\japan_life.csv')

In [16]:
# checking its shape
df2.shape

(998, 101)

In [17]:
# checking the content
df2.head(2)

,all_awardings,allow_live_comments,approved_at_utc,approved_by,archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,awarders,banned_at_utc,banned_by,can_gild,can_mod_post,category,clicked,content_categories,contest_mode,created,created_utc,crosspost_parent,crosspost_parent_list,discussion_type,distinguished,domain,downs,edited,gilded,gildings,hidden,hide_score,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,likes,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media,media_embed,media_only,mod_note,mod_reason_by,mod_reason_title,mod_reports,name,no_follow,num_comments,num_crossposts,num_reports,over_18,parent_whitelist_status,permalink,pinned,pwls,quarantine,removal_reason,report_reasons,saved,score,secure_media,secure_media_embed,selftext,selftext_html,send_replies,spoiler,steward_reports,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,title,total_awards_received,ups,url,user_reports,view_count,visited,whitelist_status,wls
0,[],False,NaN,NaN,False,suteru_away,NaN,NaN,NaN,[],NaN,NaN,NaN,text,t2_44652vdk,False,[],NaN,NaN,False,False,NaN,False,NaN,False,1.571417e+09,1.571388e+09,NaN,NaN,NaN,NaN,self.japanlife,0,False,0,{},False,False,djkknj,False,False,False,False,True,True,False,NaN,NaN,NaN,[],NaN,Medical,dark,text,False,NaN,{},False,NaN,NaN,NaN,[],t3_djkknj,False,2,0,NaN,False,all_ads,/r/japanlife/comments/djkknj/applying_for_dayc...,False,6,False,NaN,NaN,False,7,NaN,{},"Hi everyone,\n\nA question for those who have ...","&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,False,[],False,japanlife,t5_2rg2o,r/japanlife,82104,public,NaN,NaN,Applying for daycare/hoikuen,0,7,https://www.reddit.com/r/japanlife/comments/dj...,[],NaN,False,all_ads,6
1,[],False,NaN,NaN,False,Poly-be-us,NaN,NaN,NaN,[],NaN,NaN,NaN,text,t2_3o7nv8gj,False,[],NaN,NaN,False,False,NaN,False,NaN,False,1.571413e+09,1.571384e+09,NaN,NaN,NaN,NaN,self.japanlife,0,False,0,{},False,False,djk29k,False,False,False,False,True,True,False,NaN,NaN,NaN,[],NaN,NaN,dark,text,False,NaN,{},False,NaN,NaN,NaN,[],t3_djk29k,True,24,0,NaN,False,all_ads,/r/japanlife/comments/djk29k/where_to_find_obs...,False,6,False,NaN,NaN,False,3,NaN,{},I was wondering where I can find obscure old g...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,False,[],False,japanlife,t5_2rg2o,r/japanlife,82104,public,NaN,NaN,Where to find obscure old games?,0,3,https://www.reddit.com/r/japanlife/comments/dj...,[],NaN,False,all_ads,6


In [18]:
# Extracting onle title, content of the post, postid to check the duplicates and subreddit category 
japan_life_df=df2.loc[:,['id','selftext','subreddit','title']]

In [19]:
# Checking the new dataframe Japan _life
japan_life_df.head()

,id,selftext,subreddit,title
0,djkknj,"Hi everyone,\n\nA question for those who have ...",japanlife,Applying for daycare/hoikuen
1,djk29k,I was wondering where I can find obscure old g...,japanlife,Where to find obscure old games?
2,djixbc,"I want to get a new televsion, but will all th...",japanlife,Anyone got a Vizio television ? (or imported tv)
3,djifjt,Any recommendations for inside of Tokyo? I wou...,japanlife,English speaking pediatrics clinic
4,djhykp,I moved to Tokyo relatively recently and I had...,japanlife,Hobbies in Tokyo?


In [20]:
japan_life_df[japan_life_df['selftext'].isnull()]

,id,selftext,subreddit,title
25,dilj23,NaN,japanlife,Amplifier / AV Receiver options in Japan with ...
78,dgsvc8,NaN,japanlife,Earthquake! Can we get a volcano too for a ful...
87,dgp1sj,NaN,japanlife,Searching for an English Speaking therapist/co...
235,dbcamf,NaN,japanlife,Shortlisted for JSPS fellowship
294,d99u2d,NaN,japanlife,Japan residents - where do you go on holiday i...
309,d8on65,NaN,japanlife,"I’m moving away in a few months, what’s the fa..."
313,d8jrfv,NaN,japanlife,Does anyone know websites to buy Dutasteride o...
350,d77kj4,NaN,japanlife,Where do I find weather reports written by peo...
401,d5gij2,NaN,japanlife,Does anyone know any maternity hospital that h...
405,d5capi,NaN,japanlife,Applied for PR - then moved house. Anyone know...


In [21]:
# fill with single space for NaN to retain the title after title and selftest concatenation
japan_life_df['selftext']=japan_life_df['selftext'].fillna(' ')

In [22]:
# checkins for duplicates
japan_life_df.duplicated().sum()

0

In [23]:
# checking the shape
japan_life_df.shape

(998, 4)

##### We have 998 posts which are good enough for our classification task

In [24]:
# joining title andcontent of subreddit with a single space
japan_life_df['content']=japan_life_df['title']+" "+japan_life_df['selftext']


In [25]:
# dropping all other columns other than content and subreddit
japan_life_df=japan_life_df.drop(columns=['title','selftext','id'])
japan_life_df=japan_life_df.reindex(columns=['content','subreddit'])
japan_life_df.head()
# Checking the dataframe

,content,subreddit
0,"Applying for daycare/hoikuen Hi everyone,\n\nA...",japanlife
1,Where to find obscure old games? I was wonderi...,japanlife
2,Anyone got a Vizio television ? (or imported t...,japanlife
3,English speaking pediatrics clinic Any recomme...,japanlife
4,Hobbies in Tokyo? I moved to Tokyo relatively ...,japanlife


In [26]:
# concatenating Japan travel and Japan life dataframe
japan_df=pd.concat([japan_travel_df, japan_life_df], ignore_index=True,axis=0)


In [27]:
# mapping the subreddit column to 0 for japan travel and 1 for japan life
japan_df['subreddit']=japan_df['subreddit'].map({'JapanTravel':0,'japanlife':1})

In [28]:
# checking the dataframe for top 5 rows which is related to japan travel
japan_df.head()

,content,subreddit
0,Me and my travel-buddy really want to catch a ...,0
1,Ishigaki/Okinawa Just wondering where the best...,0
2,PSA: Tobu-Nikko service resumption Anyone else...,0
3,Did I Book the Osaka Cup Ramen Museum Correctl...,0
4,Tokyo motorcycle route Thinking of renting a m...,0


In [29]:
# checking the dataframe for bottom 5 rows which is related to japan life
japan_df.tail()

,content,subreddit
1989,Any experiences directly renting to tenants wi...,1
1990,SoftBank iPhone 7 unlock? Moving to Canada As ...,1
1991,Mosquito bites: how to survive Mosquito bites ...,1
1992,Modem for connecting router to apartment inter...,1
1993,Has anybody been requested by Japan Post bank ...,1


In [30]:
# checking the data types
japan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 2 columns):
content      1994 non-null object
subreddit    1994 non-null int64
dtypes: int64(1), object(1)
memory usage: 31.2+ KB


### Content column is string(object) and subreddit is integer value. This data types are good for our classification. So, no need to change the datatype.

In [31]:
# initialising lemmatizer
lemmatizer = WordNetLemmatizer()


In [32]:
# initialising stemmer
ps = PorterStemmer()


In [33]:
# function to do round 1 cleaning
def clean_round1(text):
    text = re.findall('[A-Za-z]{2,}-[A-Za-z]+|[A-Za-z]+|[A-Z]{2,}',text)
    # [A-Za-z]{2,}-[A-Za-z]+ this regex finds the words split by '-'
    # [A-Za-z]+ this regex finds the words 
    # [A-Z]{2,} this regex finds the 2 or more capital letterstogether
    # The result of text is a list
    text = [x.lower() for x in text]
    # convert all the text collected from regex into lower case in a list using list comprehension
    text = [w for w in text if not w in stopwords.words('english')]
    # write all words other than stop words into a list
    return text 
    # this function return list of words under text as a result

In [34]:
# appling the function to the content column in japan_df dataframe
japan_df['clean_round1']=japan_df['content'].apply(lambda x: clean_round1(x))

In [35]:
# function to lemmatize
def lemma(text):
    text = [lemmatizer.lemmatize(x) for x in text]
    # lemmatize function is written as list comprehension
    return text
    # our output is a list od lemmatized words


In [36]:
# application of the lemmatize function to the result from clean_round1 function and converting
# into a column in the Japan_df dataframe
japan_df['clean_lemma']=japan_df['clean_round1'].apply(lambda x: lemma(x))

In [37]:
# function for stemming
def stem(text):
    text = [ps.stem(x) for x in text]
    return text  
# result will be a list of stemmed words as stem function in list comprehension

In [38]:
# applying stem function as lambda function to the round1 cleaned text and converting into a 
# column in the Japan_df dataframe
japan_df['clean_stem']=japan_df['clean_round1'].apply(lambda x: stem(x))


In [39]:
# applying stem function to the lematized words and converting into a column in the Japan_df dataframe
japan_df['clean_lemma_stem']=japan_df['clean_lemma'].apply(lambda x: stem(x))

In [40]:
# checking the data frame for 3 new columns
japan_df

,content,subreddit,clean_round1,clean_lemma,clean_stem,clean_lemma_stem
0,Me and my travel-buddy really want to catch a ...,0,"[travel-buddy, really, want, catch, mma-fight,...","[travel-buddy, really, want, catch, mma-fight,...","[travel-buddi, realli, want, catch, mma-fight,...","[travel-buddi, realli, want, catch, mma-fight,..."
1,Ishigaki/Okinawa Just wondering where the best...,0,"[ishigaki, okinawa, wondering, best, places, s...","[ishigaki, okinawa, wondering, best, place, sn...","[ishigaki, okinawa, wonder, best, place, snork...","[ishigaki, okinawa, wonder, best, place, snork..."
2,PSA: Tobu-Nikko service resumption Anyone else...,0,"[psa, tobu-nikko, service, resumption, anyone,...","[psa, tobu-nikko, service, resumption, anyone,...","[psa, tobu-nikko, servic, resumpt, anyon, els,...","[psa, tobu-nikko, servic, resumpt, anyon, els,..."
3,Did I Book the Osaka Cup Ramen Museum Correctl...,0,"[book, osaka, cup, ramen, museum, correctly, m...","[book, osaka, cup, ramen, museum, correctly, m...","[book, osaka, cup, ramen, museum, correctli, m...","[book, osaka, cup, ramen, museum, correctli, m..."
4,Tokyo motorcycle route Thinking of renting a m...,0,"[tokyo, motorcycle, route, thinking, renting, ...","[tokyo, motorcycle, route, thinking, renting, ...","[tokyo, motorcycl, rout, think, rent, motor, c...","[tokyo, motorcycl, rout, think, rent, motor, c..."
5,"Second time visiting Japan, would love some fe...",0,"[second, time, visiting, japan, would, love, f...","[second, time, visiting, japan, would, love, f...","[second, time, visit, japan, would, love, feed...","[second, time, visit, japan, would, love, feed..."
6,Change consolidation Is there a machine or pla...,0,"[change, consolidation, machine, place, japan,...","[change, consolidation, machine, place, japan,...","[chang, consolid, machin, place, japan, consol...","[chang, consolid, machin, place, japan, consol..."
7,Need help with planning a day trip Mt. Takao H...,0,"[need, help, planning, day, trip, mt, takao, h...","[need, help, planning, day, trip, mt, takao, h...","[need, help, plan, day, trip, mt, takao, hi, t...","[need, help, plan, day, trip, mt, takao, hi, t..."
8,"which Toyota museum to see F1 model? すみません, s...",0,"[toyota, museum, see, f, model, want, visit, t...","[toyota, museum, see, f, model, want, visit, t...","[toyota, museum, see, f, model, want, visit, t...","[toyota, museum, see, f, model, want, visit, t..."
9,Was planning to bike to Arashiyama Monkey Park...,0,"[planning, bike, arashiyama, monkey, park, ren...","[planning, bike, arashiyama, monkey, park, ren...","[plan, bike, arashiyama, monkey, park, rental,...","[plan, bike, arashiyama, monkey, park, rental,..."


In [42]:
# data frame is pickled to be used in another jupiter notebook as this file have a long processing 
# time
japan_df.to_pickle("../datasets/japan_cleaned_data.pkl")

### Most of our preprocessing is done except vectorising and splitting into test and train sets. In our next jupiter notebook, we will choose our corpus, vectorise, EDA, model and evaluate the model